# Train Models

The central goal of machine learning is to train predictive models that can be used by applications. In Azure Machine Learning,  you can use scripts to train models leveraging common machine learning frameworks like Scikit-Learn, Tensorflow, PyTorch, SparkML, and others. You can run these training scripts as experiments in order to track metrics and outputs, which include the trained models.

## Connect to your workspace

To get started, connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.32.0 to work with winequality


## Create a training script

You're going to use a Python script to train a machine learning model based on the diabates data, so let's start by creating a folder for the script and data files.

In [2]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

'diabetes-training/diabetes.csv'

Now you're ready to create the training script and save it in the folder.

> **Note**: This code *creates* the script - it doesn't run it!

In [3]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing diabetes-training/diabetes_training.py


## Run the training script as an experiment

Now you're ready to run the script as an experiment. Note that the default environment does not include the **scikit-learn** package, so you need to explicitly add that to the configuration. The conda environment is built on-demand the first time the experiment is run, and cached for future runs that use the same configuration; so the first run will take a little longer.

In [4]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                environment=env) 

# submit the experiment run
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mslearn-train-diabetes_1628433800_00e9d32c',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-08T14:43:24.661306Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '564d7f93-309c-4cf9-8a52-ff5d6e4f89fd',
  'azureml.git.repository_uri': 'https://github.com/pmathur20/mslearn-dp100',
  'mlflow.source.git.repoURL': 'https://github.com/pmathur20/mslearn-dp100',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '1136edad94bcd8f9356ee4c7eb9ea82ba578ef1d',
  'mlflow.source.git.commit': '1136edad94bcd8f9356ee4c7eb9ea82ba578ef1d',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'dat

KeyError: 'log_files'

You can retrieve the metrics and outputs from the **Run** object.

In [5]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.01
Accuracy 0.774
AUC 0.8484929598487486


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/17289_azureml.log
outputs/diabetes_model.pkl


## Register the trained model

Note that the outputs of the experiment include the trained model file (**diabetes_model.pkl**). You can register this model in your Azure Machine Learning workspace, making it possible to track model versions and retrieve them later.

In [6]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 3
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774




> **Note**: The code above uses *properties* and *tags* to note additional information about the model. The key difference between properties and tags is that properties cannot be changed after the model is registered, while tags for a registered model can be modified.

## Create a parameterized training script

You can increase the flexibility of your training experiment by adding parameters to your script, enabling you to repeat the same training experiment with different settings. In this case, you'll add a parameter for the regularization rate used by the logistic regression algorithm when training the model.

Again, lets start by creating a folder for the parameterized script and the training data.

In [7]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

'diabetes-training-params/diabetes.csv'

Now let's create a script with an argument for the regularization rate hyperparameter. The argument is read using a Python **argparse.ArgumentParser** object.

In [8]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the diabetes dataset
print("Loading Data...")
# load the diabetes dataset
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing diabetes-training-params/diabetes_training.py


## Run the script with arguments

You run the script as an experiment like you did previously, reusing the environment you created; but this time you must provide the **--reg_rate** parameter that the script expects as an argument.

In [9]:
# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=env) 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mslearn-train-diabetes_1628433875_02ba8e74',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-08T14:44:40.615662Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '7b090ea8-f8d5-46de-a2dd-2a1a02a0f34a',
  'azureml.git.repository_uri': 'https://github.com/pmathur20/mslearn-dp100',
  'mlflow.source.git.repoURL': 'https://github.com/pmathur20/mslearn-dp100',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '1136edad94bcd8f9356ee4c7eb9ea82ba578ef1d',
  'mlflow.source.git.commit': '1136edad94bcd8f9356ee4c7eb9ea82ba578ef1d',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg_rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outp

Once again, we can get the metrics and outputs from the completed run.

In [10]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.774
AUC 0.8483198169063138


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/17390_azureml.log
outputs/diabetes_model.pkl


## Register a new version of the model

Now that you've trained a new model, you can register it as a new version in the workspace.

In [11]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 4
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 3
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774




**Inference Section**

**Create the Inference Script**

You can also view registered models in your workspace on the **Models** page in [Azure Machine Learning studio](https://ml.azure.com).

If you've finished exploring, you can close this notebook and shut down your compute instance.

In [12]:
%%writefile $training_folder/score.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    log_txt = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_txt)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

Writing diabetes-training-params/score.py


**Create the Dependencies for the Inference Script**

In [13]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = './diabetes-training-params/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

Saved dependency info in ./diabetes-training-params/env.yml


**Create an Inference Configuration**

In [14]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = './diabetes-training-params',
                                              entry_script="score.py",
                                              conda_file="env.yml")

**Create the Inference Cluster**

In [15]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

InProgress........................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [16]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

**Deploy the Model to the Inference Cluster**

In [46]:
from azureml.core.model import Model

model = ws.models['diabetes_model']
service = Model.deploy(workspace=ws,
                       name = 'diabetes-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-08 19:15:56+00:00 Creating Container Registry if not exists.
2021-08-08 19:16:00+00:00 Use the existing image..
2021-08-08 19:16:03+00:00 Creating resources in AKS.
2021-08-08 19:16:04+00:00 Submitting deployment to compute.
2021-08-08 19:16:04+00:00 Checking the status of deployment diabetes-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: ec907c22-547a-4dac-9f47-491d0c259e4b
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'azureml.api', please run print(service.get_logs()) to get details."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#deb

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: ec907c22-547a-4dac-9f47-491d0c259e4b
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'azureml.api', please run print(service.get_logs()) to get details."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/

In [47]:
print(ws.webservices)

# Choose the webservice you are interested in

from azureml.core import Webservice

service = Webservice(ws, 'diabetes-service')
print(service.get_logs())

{'diabetes-service': AksWebservice(workspace=Workspace.create(name='winequality', subscription_id='43871136-0ee1-4ac6-9cf7-57f9f7a83439', resource_group='rg-ml'), name=diabetes-service, image_id=None, compute_type=None, state=AKS, scoring_uri=None, tags=http://52.172.236.64:80/api/v1/service/diabetes-service/score, properties={}, created_by={'azureml.git.repository_uri': 'https://github.com/pmathur20/mslearn-dp100', 'mlflow.source.git.repoURL': 'https://github.com/pmathur20/mslearn-dp100', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': '1136edad94bcd8f9356ee4c7eb9ea82ba578ef1d', 'mlflow.source.git.commit': '1136edad94bcd8f9356ee4c7eb9ea82ba578ef1d', 'azureml.git.dirty': 'True', 'hasInferenceSchema': 'False', 'hasHttps': 'False'})}
2021-08-08T19:17:08,484773011+00:00 - iot-server/run 
2021-08-08T19:17:08,486449821+00:00 - rsyslog/run 
2021-08-08T19:17:08,487512928+00:00 - nginx/run 
2021-08-08T19:17:08,492442457+00:00 - gunicorn/run 
File not fou

In [43]:
#!pip install azure
#!pip install azure-ml-api-sdk --Done



ERROR: Could not find a version that satisfies the requirement azure.ml (from versions: none)
ERROR: No matching distribution found for azure.ml


In [35]:
service.update(enable_app_insights=True)

**Perform predictions with the Inference Cluster**

In [36]:
endpoint = service.scoring_uri
print(endpoint)

http://52.172.236.64:80/api/v1/service/diabetes-service/score


In [37]:
primary_key, secondary_key = service.get_keys()
primary_key

'sIKV9iGWWrzX2vrvb3QCuCUMunfiNpHD'

In [38]:
import requests
import json

# An array of new data cases
x_new = [[1883350,9,103,78,25,304,29.58219193,1.282869847,43]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + primary_key }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

print(response)

<Response [503]>


In [39]:
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [50]:
import sklearn
sklearn.__version__

'0.22.2.post1'